In [12]:
class Puzzle:
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,0]
    heuristic = None
    evaluation_function = None
    needs_heuristic = False
    num_of_instances = 0

    def __init__(self, state, parent, action, path_cost, needs_heuristic=False,h1=False):
        self.parent = parent
        self.state = state
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        if needs_heuristic:
            self.needs_heuristic = True
            if h1:
                self.generate_heuristic_h1()
            else:
                self.generate_heuristic_h2()
            self.evaluation_function = self.heuristic + self.path_cost
        Puzzle.num_of_instances += 1

    def __str__(self):
        return str(self.state[0:4]) + '\n' + str(self.state[4:8]) + '\n' + str(self.state[8:12]) + '\n' + str(self.state[12:16])

    def generate_heuristic_h2(self):
        self.heuristic = 0
        for num in range(1, 16):
            if self.state.index(num) != self.goal_state.index(num):
                distance = abs(self.state.index(num) - self.goal_state.index(num))
                i = int(distance / 4)
                j = int(distance % 4)
                self.heuristic += i + j

    def generate_heuristic_h1(self):
        self.heuristic = 0
        for num in range(1, 16):
            if self.state.index(num) != self.goal_state.index(num):
                self.heuristic += 1  # Count each misplaced tile

#     def generate_heuristic(self):
#         self.heuristic = 0
#         for num in range(1, 16):
#             if self.state.index(num) != self.goal_state.index(num):
#                 distance = abs(self.state.index(num) - self.goal_state.index(num))
#                 i = int(distance / 4)
#                 j = int(distance % 4)
#                 self.heuristic += abs(i - int((num - 1) / 4)) + abs(j - (num - 1) % 4)


    def goal_test(self):
        return self.state == self.goal_state

    @staticmethod
    def find_legal_actions(i, j):
        legal_actions = ['U', 'D', 'L', 'R']
        if i == 0:
            legal_actions.remove('U')
        elif i == 3:
            legal_actions.remove('D')
        if j == 0:
            legal_actions.remove('L')
        elif j == 3:
            legal_actions.remove('R')
        return legal_actions

    def generate_child(self):
        children = []
        x = self.state.index(0)
        i = int(x / 4)
        j = int(x % 4)
        legal_actions = self.find_legal_actions(i, j)

        for action in legal_actions:
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x - 4] = new_state[x - 4], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x + 4] = new_state[x + 4], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x - 1] = new_state[x - 1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x + 1] = new_state[x + 1], new_state[x]
            child=Puzzle(new_state, self, action, 1, self.needs_heuristic)
            children.append(child)
        return children

    def find_solution(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent is not None:
            path = path.parent
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution



In [13]:
import random
def generate_random_state():
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,0]
    puzzle=Puzzle(goal_state.copy(),None,None,0)
    for i in range(50):
        puzzle=random.choice(puzzle.generate_child())
    return puzzle.state




In [14]:
# implement BFS
from queue import Queue

def breadth_first_search(initial_state):
    start_node = Puzzle(initial_state, None, None, 0)
    if start_node.goal_test():
        return start_node.find_solution()
    
    q = Queue()
    q.put(start_node)
    explored = []

    while not q.empty():
        node = q.get()
        explored.append(node.state)
        children = node.generate_child()

        for child in children:
            if child.state not in explored:
                if child.goal_test():
                    return child.find_solution()
                q.put(child)
    
    return



In [15]:
# implement DFS
def depth_first_search(initial_state):
    start_node = Puzzle(initial_state, None, None, 0)
    if start_node.goal_test():
        return start_node.find_solution()
    
    opened =[]
    opened.append(start_node)
    explored = set()

    while opened:
        node = opened.pop()
        explored.add(tuple(node.state))
        children = node.generate_child()

        for child in children:
            if tuple(child.state) not in explored:
                if child.goal_test():
                    return child.find_solution()
                opened.append(child)
    
    return



In [16]:
# implement AStar with h1
from queue import PriorityQueue

def Astar_search_h1(initial_state):
    count = 0
    explored = []
    start_node = Puzzle(initial_state, None, None, 0, True,True)
    q = PriorityQueue()
    q.put((start_node.evaluation_function, count, start_node))

    while not q.empty():
        node = q.get()
        node = node[2]
        explored.append(node.state)
        if node.goal_test():
            return node.find_solution()

        children = node.generate_child()
        for child in children:
            if child.state not in explored:
                count += 1
                q.put((child.evaluation_function, count, child))
    return


In [17]:
# implement AStar with h2
from queue import PriorityQueue

def Astar_search_h2(initial_state):
    count = 0
    explored = []
    start_node = Puzzle(initial_state, None, None, 0, True,False)
    q = PriorityQueue()
    q.put((start_node.evaluation_function, count, start_node))

    while not q.empty():
        node = q.get()
        node = node[2]
        explored.append(node.state)
        if node.goal_test():
            return node.find_solution()

        children = node.generate_child()
        for child in children:
            if child.state not in explored:
                count += 1
                q.put((child.evaluation_function, count, child))
    return


In [18]:
from time import time
state=generate_random_state()


puzzle_=Puzzle(state,None,None,0,True,True)
print("state :\n",puzzle_.state)
print("h1: ",puzzle_.heuristic)
puzzle_=Puzzle(state,None,None,0,True,False)

print("h2: ",puzzle_.heuristic)



state :
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 11, 12, 13, 15, 10, 0]
h1:  3
h2:  4


In [19]:

# Astar with h1 :Number of Tiles Out of Place
Puzzle.num_of_instances=0
t0=time()
astar=Astar_search_h1(state)
t1=time()-t0
print('astar(h1):', astar)
print('space:',Puzzle.num_of_instances)
print('time:',t1)




astar(h1): ['U', 'L', 'D', 'L', 'U', 'R', 'R', 'D']
space: 57
time: 0.0


In [20]:

# Astar with h2 :Manhattan Distance
Puzzle.num_of_instances=0
t0=time()
astar=Astar_search_h2(state)
t1=time()-t0
print('astar(h2):', astar)
print('space:',Puzzle.num_of_instances)
print('time:',t1)




astar(h2): ['U', 'L', 'D', 'L', 'U', 'R', 'R', 'D']
space: 57
time: 0.008120298385620117


In [21]:

# bfs

Puzzle.num_of_instances=0
t0=time()
bfs=breadth_first_search(state)
t1=time()-t0
print('BFS:', bfs)
print('space:',Puzzle.num_of_instances)
print('time:',t1)

BFS: ['U', 'L', 'D', 'L', 'U', 'R', 'R', 'D']
space: 864
time: 0.010564327239990234


In [22]:
from time import time
# dfs
state=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 13, 14, 15, 12]
state=[1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]
#  [1, 2, 3, 4, 5, 0, 6, 7, 9, 10, 11, 8, 13, 14, 15, 12]

# state=generate_random_state_dfs()
Puzzle.num_of_instances=0
t0=time()
dfs=depth_first_search(state)
t1=time()-t0
print('dfs:', dfs)
print('space:',Puzzle.num_of_instances)
print('time:',t1)




dfs: ['L', 'L', 'L', 'D', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'U', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'D', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R',